#### Name: Anush Ravishankar
#### Registration No: 200968250
##### Week - 8 AI 
##### Section: A -- Roll No: 61

In [ ]:
!pip install gym

import gym
import numpy as np
env = gym.make("FrozenLake-v1")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


####  Policy Iteration function with the following parameters 
 - policy: 2D array of a size n(S) x n(A), each cell represents a probability of taking action 'a' in state 's'
 - environment: Initialized Open AI gym environment object
 - discount_factor: MDP discount factor
 - theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is lesser than this number
 - max_iterations: Maximum number of iterations

In [ ]:
import numpy as np

def policy_iteration(policy, env, discount_factor, theta, max_iterations):
    # Get number of states and actions
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    
    # Initialize value function with zeros
    value_function = np.zeros(num_states)
    
    # Iterate until convergence or maximum number of iterations is reached
    for i in range(max_iterations):
        
        # Policy Evaluation
        while True:
            # Initialize the maximum change in value function for the current iteration
            delta = 0
            
            # Iterate over all states
            for s in range(num_states):
                # Save the old value for the current state
                old_value = value_function[s]
                
                # Get the action probabilities for the current state according to the current policy
                action_prob = policy[s]
                
                # Initialize the action values for the current state and action
                action_values = np.zeros(num_actions)
                
                # Iterate over all actions for the current state
                for a in range(num_actions):
                    # Iterate over all possible transitions for the current state-action pair
                    for prob, next_state, reward, done in env.P[s][a]:
                        # Calculate the action value for the current state-action pair
                        action_values[a] += prob * (reward + discount_factor * value_function[next_state])
                
                # Update the value function for the current state using the action values and action probabilities
                value_function[s] = np.sum(action_prob * action_values)
                
                # Calculate the maximum change in value function for the current iteration
                delta = max(delta, np.abs(old_value - value_function[s]))
            
            # If the maximum change in value function is less than the threshold theta, break the loop
            if delta < theta:
                break
        
        # Policy Improvement
        policy_stable = True
        
        # Iterate over all states
        for s in range(num_states):
            # Save the old action for the current state
            old_action = np.argmax(policy[s])
            
            # Initialize the action values for the current state and action
            action_values = np.zeros(num_actions)
            
            # Iterate over all actions for the current state
            for a in range(num_actions):
                # Iterate over all possible transitions for the current state-action pair
                for prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the action value for the current state-action pair
                    action_values[a] += prob * (reward + discount_factor * value_function[next_state])
            
            # Choose the best action for the current state based on the action values
            best_action = np.argmax(action_values)
            
            # If the old action is not equal to the new best action, set policy_stable to False
            if old_action != best_action:
                policy_stable = False
            
            # Update the policy for the current state based on the best action
            policy[s] = np.eye(num_actions)[best_action]
        
        # If the policy is stable, break the loop
        if policy_stable:
            break
    
    # Return the final policy and value function
    return policy, value_function


#### Value Iteration function with the following parameters 
 - environment: Initialized Open AI gym environment object 
 - discount_factor: MDP discount factor 
 - theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is below this number
 - max_iterations: Maximum number of iterations

In [ ]:
def value_iteration(env, discount_factor, theta, max_iterations):
    # Get the number of states and actions from the environment
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    
    # Initialize the value function with zeros for all states
    value_function = np.zeros(num_states)
    
    # Iterate until the value function converges or the maximum number of iterations is reached
    for i in range(max_iterations):
        delta = 0
        # For each state, find the action that maximizes the expected return
        for s in range(num_states):
            old_value = value_function[s]
            action_values = np.zeros(num_actions)
            # For each action, compute the expected return by summing over all possible next states and rewards
            for a in range(num_actions):
                for prob, next_state, reward, done in env.P[s][a]:
                    action_values[a] += prob * (reward + discount_factor * value_function[next_state])
            # Update the value function for the current state by taking the maximum over all actions
            value_function[s] = np.max(action_values)
            # Keep track of the maximum change in the value function
            delta = max(delta, np.abs(old_value - value_function[s]))
        # If the maximum change is less than the threshold, break out of the loop
        if delta < theta:
            break
    
    # Compute the policy by choosing the action that maximizes the expected return for each state
    policy = np.zeros((num_states, num_actions))
    for s in range(num_states):
        action_values = np.zeros(num_actions)
        for a in range(num_actions):
            for prob, next_state, reward, done in env.P[s][a]:
                action_values[a] += prob * (reward + discount_factor * value_function[next_state])
        # Choose the action that maximizes the expected return
        best_action = np.argmax(action_values)
        # Update the policy with the chosen action
        policy[s][best_action] = 1.0
    
    # Return the policy and the final value function
    return policy, value_function



#### Compare  the 
 - number of  wins
 - average  return  after  1000  episodes 

In [ ]:
def run_episodes(policy, env, num_episodes):
    total_reward = 0
    num_wins = 0
    for i in range(num_episodes):
        state = env.reset()
        episode_reward = 0
        done = False
        while not done:
            action = np.random.choice(env.action_space.n, p=policy[state])
            next_state, reward, done, _ = env.step(action)
            episode_reward += reward
            state = next_state
        total_reward += episode_reward
        if episode_reward == 1:
            num_wins += 1
    return num_wins, total_reward / num_episodes

n_states = env.observation_space.n
n_actions = env.action_space.n

# Set the parameters
discount_factor = 0.99
theta = 1e-8
max_iterations = 2000
num_episodes = 1000

# Run Policy Iteration
policy = np.ones([n_states, n_actions]) / n_actions
opt_policy, value_func = policy_iteration(policy, env, discount_factor, theta, max_iterations)
num_wins_policy, avg_return_policy = run_episodes(opt_policy, env, num_episodes)

# Run Value Iteration
opt_policy, value_func = value_iteration(env, discount_factor, theta, max_iterations)
num_wins_value, avg_return_value = run_episodes(opt_policy, env, num_episodes)

# Print the results
print(f"Policy Iteration: Number of wins = {num_wins_policy}, Average Return = {avg_return_policy}")
print(f"Value Iteration: Number of wins = {num_wins_value}, Average Return = {avg_return_value}")


Policy Iteration: Number of wins = 746, Average Return = 0.746
Value Iteration: Number of wins = 736, Average Return = 0.736


### Inference: 
#### The results show that both Policy Iteration and Value Iteration successfully learned the optimal policy for the FrozenLake-v1 environment, as evidenced by their high number of wins and average return close to 1.

#### However, the Policy Iteration method performed slightly better in terms of the number of wins and average return. This could be because Policy Iteration directly optimizes the policy and updates the value function accordingly, while Value Iteration updates the value function directly and then extracts the policy from it. This may lead to suboptimal policies in some cases.

#### Overall, both methods are effective for solving the FrozenLake-v1 environment, but Policy Iteration may be a slightly better choice for this specific problem.